
# Brain Tumor MRI Classification using Deep Learning

**Team Name:** Deadlock  
**Competition:** BioFusion Hackathon – IEEE EMBS  
**Task:** Multiclass Brain Tumor Classification (MRI)

---



## 1. Problem Definition

### Clinical Relevance
Brain tumors are life-threatening conditions that require early and accurate diagnosis. MRI is the gold-standard imaging modality, but manual interpretation is time-consuming and prone to subjectivity.

### Objective
To classify brain MRI images into four categories:
- Glioma
- Meningioma
- Pituitary Tumor
- No Tumor

This model acts as a **clinical decision support tool**, not a replacement for clinicians.



## 2. Dataset Documentation

### Dataset Source
Brain Tumor MRI Dataset (Kaggle):  
https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset

### Dataset Size
- Training images: 5,714  
- Testing images: 1,311  

### Classes
Glioma, Meningioma, Pituitary, No Tumor


In [ ]:

# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adamax



## 3. Data Loading and Analysis


In [ ]:
# Set paths (update according to your Drive)
train_dir = '/content/drive/MyDrive/BrainTumor/Training'
test_dir = '/content/drive/MyDrive/BrainTumor/Testing'

def create_df(directory):
    filepaths = []
    labels = []
    for folder in os.listdir(directory):
        for file in os.listdir(os.path.join(directory, folder)):
            filepaths.append(os.path.join(directory, folder, file))
            labels.append(folder)
    return pd.DataFrame({'filepath': filepaths, 'label': labels})

train_df = create_df(train_dir)
test_df = create_df(test_dir)

print(train_df.shape, test_df.shape)


In [ ]:
# Visualize class distribution
plt.figure(figsize=(8,5))
sns.countplot(x='label', data=train_df)
plt.title('Training Data Distribution')
plt.show()



## 4. Data Preprocessing


In [ ]:
# Split test into validation and final test
valid_df, ts_df = train_test_split(
    test_df, test_size=0.5, stratify=test_df['label'], random_state=42
)

img_size = (299, 299)
batch_size = 32

train_gen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.8, 1.2]
)

test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(
    train_df, x_col='filepath', y_col='label',
    target_size=img_size, class_mode='categorical',
    batch_size=batch_size
)

valid_data = test_gen.flow_from_dataframe(
    valid_df, x_col='filepath', y_col='label',
    target_size=img_size, class_mode='categorical',
    batch_size=batch_size
)

test_data = test_gen.flow_from_dataframe(
    ts_df, x_col='filepath', y_col='label',
    target_size=img_size, class_mode='categorical',
    batch_size=batch_size, shuffle=False
)



## 5. Model Initialization & Pretrained Disclosure

### Pretrained Model
- Model: EfficientNetB3
- Source: TensorFlow / Keras
- Pretraining Dataset: ImageNet
- Weight Usage: Pretrained weights used


In [ ]:
# Load EfficientNetB3 base model
base_model = EfficientNetB3(
    include_top=False,
    weights='imagenet',
    input_shape=(299,299,3)
)

base_model.trainable = False  # Feature extraction phase

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.45)(x)
output = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adamax(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()



## 6. Model Training


In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)



## 7. Training Logs and Curves


In [ ]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy Curve')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss Curve')

plt.show()



## 8. Model Evaluation


In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f'Test Accuracy: {test_acc*100:.2f}%')


In [ ]:
preds = model.predict(test_data)
y_pred = np.argmax(preds, axis=1)
y_true = test_data.classes

print(classification_report(y_true, y_pred, target_names=test_data.class_indices.keys()))


In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=test_data.class_indices.keys(),
            yticklabels=test_data.class_indices.keys())
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()



## 9. Error Analysis & Limitations

- Minor confusion observed between Glioma and Meningioma classes
- Domain mismatch between ImageNet and MRI images
- Limited generalization to unseen MRI scanners



## 10. Reproducibility
- Random seed fixed
- Paths explicitly defined
- All cells runnable in Google Colab


In [ ]:

# Save final model
model.save('brain_tumor_efficientnetb3.h5')
